In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.vision import *
from fastai.basics import *
import pandas as pd
import numpy as np
import math

In [4]:
newdata = pd.read_csv("mnist_data.csv")
newdata
mydata = newdata.iloc[0:1000,]
mydata = mydata.drop(["label"],axis=1)
mydata = mydata/255
print(mydata.shape)

(1000, 784)


In [5]:
image_dim = 784
nn_dim = 512 #encoder
latent_variable_dim = 2 #for mean and std dev

def values(shape):
    val = np.random.normal(scale = 1/(np.sqrt(shape[0]/2)), size = shape)
    return (val)

In [6]:
#Finding initail weights and biases for every layer
weight_encoder = values([image_dim,nn_dim])
weight_mean = values([nn_dim, latent_variable_dim])
weight_stddev = values([nn_dim, latent_variable_dim])
weight_decoder = values([latent_variable_dim,nn_dim])
weight_output = values([nn_dim,image_dim])

bias_encoder = values([nn_dim])
bias_mean = values([latent_variable_dim])
bias_stddev = values([latent_variable_dim])
bias_decoder = values([nn_dim])
bias_output = values([image_dim])

print("Weights")
print(weight_encoder.shape)
print(weight_mean.shape)
print(weight_decoder.shape)
print(weight_output.shape)

Weights
(784, 512)
(512, 2)
(2, 512)
(512, 784)


In [25]:
#Model

#Encoder layer
encoder = np.add(np.dot(mydata,weight_encoder),bias_encoder)
encoder = torch.from_numpy(encoder)
t = nn.Tanh()
encoder_layer = t(encoder)
#print(encoder_layer.shape)
#Shape [1000,512]


#Mean and Standard deviation layer
mean = np.add(np.dot(encoder_layer,weight_mean), bias_mean)
mean_layer = torch.from_numpy(mean)
stddev = np.add(np.dot(encoder_layer,weight_stddev),bias_stddev)
stddev_layer = torch.from_numpy(stddev)
#Shape (1000,2)


#latent layer
mu,sigma = 0,1
epsilon = np.random.normal(mu, sigma, size = (1000,2))
epsilon = torch.from_numpy(epsilon)
latent_layer = torch.from_numpy(stddev)
latent_layer = torch.add(mean_layer, torch.mul((torch.exp(0.5*stddev_layer)), epsilon))
#Shape [1000,2]

#Decoder layer
lat = latent_layer.numpy()
decoder = np.add((np.dot(lat,weight_decoder)), bias_decoder)
decoder = torch.from_numpy(decoder)
decoder_layer = t(decoder)
#[100,512]

#Output layer
dec = decoder_layer.numpy()
output = np.add(np.dot(dec,weight_output), bias_output)
output = torch.from_numpy(output)
output_layer = t(output)
#[1000,784]

In [26]:
#Loss funxtion

#Loss

def loss_function(input_image, output_image):
    dfl = input_image*math.log(1e-10+output_image) + (1-original_image)*math.log(1e-10 + 1 - output_image)
    dfl = torch.from_numpy(dfl)
    data_fidelity_loss = -1*torch.sum(dfl,dim =1)
    
    kldl = 1 + stddev_layer - np.square(mean_layer) - np.exp(stddev_layer)
    kldl = torch.from_numpy(kldl)
    kl_divergence_loss = -0.5*torch.sum(kldl, dim=1)
    
    loss = 0.5*(data_fidelity_loss + kl_divergence_loss)
    return loss
